```
ME72: Maestría en Métodos Cuantitativos para la Gestión y Análisis de Datos
M72109: Analisis de datos no estructurados
Universidad de Buenos Aires - Facultad de Ciencias Economicas (UBA-FCE)
Año: 2020
Profesor: Facundo Santiago
```

# Desafio: ¿Qué tan memorable es un video?

## ¿De que se trata el desafío?

Esta tarea se centra en el problema de predecir qué tan memorable es un video para los espectadores. Requiere que los participantes predigan automáticamente las puntuaciones de memorabilidad de los videos que reflejan la probabilidad de que se recuerde un video. Recibiran un extenso conjunto de datos de videos que van acompañados de anotaciones memorables, así como características extraidas previamente para facilitarles el procesamiento. Las etiquetas (labels) se ha recopilado a través de pruebas de reconocimiento y, por lo tanto, es el resultado de una medición objetiva del rendimiento de la memoria.

<img src='Docs/memorability.png' width=600 />

Creditos del desafio original:
http://www.multimediaeval.org/mediaeval2019/memorability/

Mihai Gabriel Constantin, University Politehnica of Bucharest, Romania
Bogdan Ionescu, University Politehnica of Bucharest, Romania
Claire-Hélène Demarty, Technicolor, France
Quang-Khanh-Ngoc Duong, Technicolor, France
Xavier Alameda-Pineda, INRIA, France
Mats Sjöberg, CSC, Finland

## Direcciones

Deberán entrenar modelos computacionales capaces de inferir la memorabilidad de video a partir del contenido visual. Opcionalmente, se pueden utilizar títulos descriptivos adjuntos a los videos. Los modelos se evaluarán a través de métricas de evaluación estándar utilizadas en las tareas de clasificación (correlación de Spearman)

## Preparación del ambiente

In [4]:
import pandas as pd

Descarguemos los fragmentos de codigo que se utilizan en este caso

In [ ]:
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desadio/Utils/audio_extractor.py --directory-prefix ./Utils/
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desadio/Utils/audio_featurizer.py --directory-prefix ./Utils/

## Set de datos

Descargamos el set de datos

In [2]:
!wget -N https://santiagxf.blob.core.windows.net/public/2018_MovieMemorabilityPackage.tgz --directory-prefix Data

--2020-09-20 17:01:27--  https://santiagxf.blob.core.windows.net/public/2018_MovieMemorabilityPackage.tgz
Resolving santiagxf.blob.core.windows.net (santiagxf.blob.core.windows.net)... 52.239.220.32
Connecting to santiagxf.blob.core.windows.net (santiagxf.blob.core.windows.net)|52.239.220.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1461601236 (1.4G) [application/x-compressed]
Saving to: ‘Data/2018_MovieMemorabilityPackage.tgz’

2018_MovieMemorabil 100%[===================>]   1.36G   103MB/s    in 14s     

2020-09-20 17:01:58 (96.8 MB/s) - ‘Data/2018_MovieMemorabilityPackage.tgz’ saved [1461601236/1461601236]



Descompromiremos el archivo para obtener el set de datos

In [ ]:
!tar zxvf ./Data/2018_MovieMemorabilityPackage.tgz --directory Data

## Anotaciones para el problema de memorabilidad

Cuentan con la anotaciones para el problema de memorabilidad. Dependiendo de su elección pueden optar por resolver un problema de clasificación o un problema de regresión. Esto lo veran refrejado en:
 - **memorability_score:** Representa el puntaje de memorabilidad de la secuencia en particular. Valores altos son mejors.
 - **memorable:** Variable categórica que representa si un video es memorable o no. Un video con Score superior a 0.5 es marcado como memorable (1), sino es marcado como no memorable (0)

In [14]:
labels = pd.read_csv('Data/ground_truth.csv')

In [15]:
labels.head(2)

,movie_name,start(sec),end(sec),sequence_name,Neutral (1)_Typical (0),nb_annotations,memorability_score,memorabable
0,127 hours,2000,2010,127_hours_2000_2010_1,0,5,1.0,1
1,127 hours,2182,2192,127_hours_2182_2192_5,1,8,0.0,0


## Elije tu propia aventura

En este desadio, les propongo elegir la dificultad con la que quieren trabajar para solucionar el problema en cuestión. Dependiendo de la elección, son las herramientas que tendrán a su disosición para resolver el problema. Pueden utilizar algunas de las sugerencias, todas las sugerencias, o si conocen alguna técnica en particular que les gustaría revisar si aplica... probarla.

### Resolver el desafio utilizando los datos de origen

Esta configuración del desafio más similar a la vida real, y solo contaran como datos de entrada un conjunto de videos en formato MP4 juntos con sus anotaciones en formato CSV. Cualquier procesamiento de datos deberá realizarse utilizando esta información solamente. Como algunos procesamientos pueden ser bastante complejos, contarán además con:
 - Imágenes extraidas cada 2 segundos por cada secuencia a analizar
 - Tracks de audio disponibles en formato WAV listos para utilizar

#### Videos

En el directorio */Data/Raw/videos* encontraran todas las secuencias utilizadas en este desafio

#### Audios

En el directorio */Data/Raw/audios* encontraran todos los audios correspondientes a cada una de las secuencias de los videos. Los audios están en formato wav, que si bien ocupan mayor espacio, son más sencillos de procesar

#### Frames

En el directorio */Data/Raw/frames* encontraran todos los frames extraidos de cada una de las secuencias. Los frames están disponibles cada 2 segundos por lo cual disponen de 5 frames por cada secuencia.

### Resolver el desafío utilizando features generadas

En esta configuración del desafio, y hay bastante trabajo realizado y se les entrega en formatos CSV para que no tengan que trabajar con grandes volumenes de datos no estructurados. En particular tendran:

#### Descripciónes de las imagenes

Cuentan con las descripciones de cuadros (frames) de cada una de las secuencias. Estos frames están extraidos cada 1 segundo y las descripciones se generaron utilizando la API de Computer Vision de Azure Cognitive Services. Cada una de las descripciones forma una oración, y por lo tanto, por cada secuencia tendrán disponible un parrafo con 10 oraciones describiendo cada una de las escenas

In [5]:
captions = pd.read_csv('Data/Features/caption_features.csv')

In [6]:
captions.head(2)

,sequence_name,cc
0,127_hours_2000_2010_1,a man in a suit and tie standing in front of a...
1,127_hours_2182_2192_5,a man holding a toothbrush in his mouth. a wom...


#### Vectores de audio

Se corresponden a las representaciones (embeddings) utilizados por la red YAMNet que vimos en el curso. YAMNet es un modelo que explota esta caracteristica al construir imagenes a partir de audio (llamados parches o patchs). Cada uno de estos patchs es el resultado de calcular espectrogramas log-mel, creando así parches de imágenes 2D para utilizar en nuestro modelo.



In [8]:
audio_embeddings = pd.read_csv('Data/Features/audio_vectors.csv')

In [9]:
audio_embeddings.head(2)

,Unnamed: 0,sequence_name,1,2,3,4,5,6,7,8,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024
0,0,127_hours_2000_2010_1,0.0,0.796399,0.009879,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.492673,0.0,0.0,0.0,0.391332,0.0,0.0
1,1,127_hours_2000_2010_1,0.0,0.000000,0.000000,0.003163,0.010787,0.0,0.0,0.0,...,0.0,0.0,0.0,0.173500,0.0,0.0,0.0,0.177809,0.0,0.0


#### Vectores de la red C3D (convoluciones en 3D)

Se corresponden a las representaciones aprendidas al aplicar un modelo de convolución 3D sobre el input del video. Estas convoluciones no solo tienen en cuenta localidad a nivel de pixeles en las imagenes sino que también localidad a nivel espacio-temporal en el transcurso del video.

<img src='Docs/3D-timedistributed.PNG' width=500 />

Referencias sobre este modelo: https://arxiv.org/pdf/1711.11248.pdf<br />
Implementación por el equipo de Facebook AI: https://github.com/facebookresearch/VMZ

In [11]:
video_embeddings = pd.read_csv('Data/Features/c3d_vectors.csv')

In [12]:
video_embeddings.head(2)

,sequence_name,0,1,2,3,4,5,6,7,8,...,4087,4088,4089,4090,4091,4092,4093,4094,4095,Unnamed: 4097
0,127_hours_2000_2010_1,0.82588,6.3702,0.0,0.34248,0.75896,1.7976,0.1524,0.0,2.1005,...,0.0,5.6224,1.7332,2.4912,3.27780,1.7001,0.0000,1.9559,6.1765,NaN
1,127_hours_2182_2192_5,0.00000,2.2294,0.0,0.00000,1.53370,1.2082,2.6512,0.0,4.1553,...,0.0,3.8342,1.7536,1.8910,0.91983,0.0000,2.0523,1.0713,1.4447,NaN
